# Задание
Построить алгоритмы классификации для решения задачи классификации MNIST. 
1. Обучить модели (SVM, logisitic regression, decision tree, (дополнительные модели по желанию) на исходных данных;
2. Использовать алгоритмы PCA и t-SNE для снижения размерности. Какое количество компонент необходимо, чтобы описать 90% дисперсии в PCA?
3. Обучить те же модели на сниженном признаковом пространстве, Какой алгоритм снижения дает лучшие метрики точности моделей классификации? Есть ли улучшение относительно работы моделей на исходном признаковом пространстве?
4. Оцените, какое количество времени требуется для препроцессинга PCA и t-SNE.

In [1]:
# Испортируем необходимые библиотеки
import pandas as pd 
import time

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn import datasets 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE 

In [2]:
# Загружаем датасет
digits = datasets.load_digits()
X = digits.data
y = digits.target

In [3]:
# Обучим моделей SVM, logisitic regression и decision tree на исходных данных (без уменьшения размерности)

# Разделим данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    "SVM": SVC(random_state=42),
    "LogisticRegression" : LogisticRegression(max_iter=10000, random_state=42),
    "DecisionTreeClassifier": DecisionTreeClassifier(random_state=42)
}

# Получим предикты и точность моделей
print('Model Accuracy')
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'{model_name}: {accuracy:.4f}')

Model Accuracy
SVM: 0.9861
LogisticRegression: 0.9722
DecisionTreeClassifier: 0.8417


In [4]:
process_time = {}

In [5]:
# Применим алгоритмы снижения размерности PCA и t-SNE и обучим модели на данных с пониженной размерностью

# PCA
start_time = time.time()
pca = PCA(n_components=0.9)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
process_time['PCA'] = time.time() - start_time

print(f'Необходимое количество компонент для описания 90% дисперсии : {pca.n_components_}')

# Обучение моделей на сниженном признаковом пространстве 
print(f'\nModel accuracy with PCA')
for model_name, model in models.items():
    model.fit(X_train_pca, y_train)
    y_pred = model.predict(X_test_pca)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'{model_name}: {accuracy:.4f}')

Необходимое количество компонент для описания 90% дисперсии : 21

Model accuracy with PCA
SVM: 0.9861
LogisticRegression: 0.9583
DecisionTreeClassifier: 0.8667


In [6]:
X_train_tsne, X_test_tsne, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# t-SNE
start_time = time.time()
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X)
process_time['t-SNE'] = time.time() - start_time

# Обучение моделей на сниженном признаковом пространстве 
print(f'\nModel accuracy with t-SNE')
for model_name, model in models.items():
    model.fit(X_train_tsne, y_train)
    y_pred = model.predict(X_test_tsne)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name}: {accuracy:.4f}")


Model accuracy with t-SNE
SVM: 0.9861
LogisticRegression: 0.9722
DecisionTreeClassifier: 0.8417


In [7]:
# Посмотрим какое время требуется для препроцессинга PCA и t-SNE
print(f'Время, затраченнное на препроцессинг:\n')
for transformer_name, time in process_time.items():
    print(f'{transformer_name}: {time:.3f} секунд')

Время, затраченнное на препроцессинг:

PCA: 0.015 секунд
t-SNE: 8.078 секунд


## Результаты

Для сохранения 90% дисперсии в алгоритме PCA необходима 21 компонента

Для модели SVM мы не увидели изменений в точности при снижении размерности

Для модели Logistic Regression наблюдается снижение точности при использовании алгоритма PCA, при использовании алгоритма t-SNE значение точности не изменилось

Для модели Decision Tree наблюдается прирост точности для алгоритма PCA

Время работы препроцессинга для t-SNE алгоритма выше:

- PCA : 0.015 секунд
- t-SNE : 8.078 секунд